In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
tf.__version__     

'2.11.0'

In [6]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [16]:
X=df.iloc[:,3:-1].values
y=df.iloc[:,-1].values
print(X)
print()
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]

[1 0 1 ... 1 1 0]


In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[("encoder",OneHotEncoder(),[1])],remainder="passthrough")
X=np.array(ct.fit_transform(X))
print(X[1])

[1.0 0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]


In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=0)

In [22]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [23]:
ann=tf.keras.models.Sequential()
#input layer
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

In [24]:
#2nd hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

In [25]:
#Output Layer

ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [27]:
#Compiling
ann.compile(optimizer= "adam",loss="binary_crossentropy",metrics=["accuracy"])


In [28]:
#Training
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5619 - accuracy: 0.7544
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4822 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4580 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4450 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4376 - accuracy: 0.7961
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4324 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4292 - accuracy: 0.7964
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4264 - accuracy: 0.7968
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4237 - accuracy: 0.8099
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4215 - accura

In [31]:
#Prediction
print(ann.predict(sc.transform([[1,0,0,1,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 26ms/step
[[False]]


In [33]:
y_pred=ann.predict(X_test)
y_pred=(y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [34]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1501   94]
 [ 193  212]]


0.8565